In [10]:
import json
import glob
import os

import numpy as np

import cv2
import k4a
import mediapipe as mp
import pyk4a

import utils.utils_mediapipe as utils_mediapipe

#### Video to frames

In [11]:
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

mp_solver_settings = dict(
    static_image_mode=False,
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)
mp_solver = mp_holistic.Holistic(**mp_solver_settings)

In [39]:
filenames = glob.glob(os.path.join('data', 'undistorted', 'G101', 'select', 'right', 'trial1', 'cam_center', 'color', '*.jpg'))
# source_filename = 'color.avi'
# save_folder = 'test_frames_data'
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder, exist_ok=True)

for source_filename in filenames:
    frame = cv2.imread(source_filename)
    landmarks = mp_solver.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if landmarks.pose_landmarks is not None:
        frame = frame.copy()
        mp_drawing.draw_landmarks(
            frame,
            landmarks.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        mp_drawing.draw_landmarks(
            frame,
            landmarks.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        mp_drawing.draw_landmarks(
            frame,
            landmarks.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
    # mp_drawing.plot_landmarks(
    #     landmarks.pose_world_landmarks,
    #     mp_pose.POSE_CONNECTIONS,
    # )
    cv2.imshow('MediaPipe Pose', frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [37]:
source_filename = 'color.avi'
save_folder = 'test_frames_data'
if not os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

cap = cv2.VideoCapture(source_filename)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    landmarks = mp_solver.process(frame)
    if landmarks.pose_landmarks is not None:
        frame = frame.copy()
        mp_drawing.draw_landmarks(
            frame,
            landmarks.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        mp_drawing.draw_landmarks(
            frame,
            landmarks.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        mp_drawing.draw_landmarks(
            frame,
            landmarks.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        # mp_drawing.plot_landmarks(
        #     landmarks.pose_world_landmarks,
        #     mp_pose.POSE_CONNECTIONS,
        # )
    cv2.imshow('MediaPipe Pose', frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

#### Video calibration params

In [7]:
filename = "output.mkv"
playback = pyk4a.PyK4APlayback(filename)

In [38]:
playback.open()
calib = playback.calibration_raw
calib_json = json.dumps(calib)
playback.close()

In [37]:
playback.open()
while True:
    try:
        frame = playback.get_next_capture()
    except EOFError as err:
        break
playback.close()

In [50]:
frame.color.shape

(279859,)

In [47]:
frame.depth.shape

(576, 640)

#### Transformation Depth -> Color

#### MediaPipe

In [51]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

mp_solver_settings = dict(
    static_image_mode=True,
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)
mp_solver = mp_pose.Pose(**mp_solver_settings)

In [56]:
mp_array = np.array([[[i * 1000 + j * 10 + k for k in range(3)] for j in range(33)] for i in range(120)])

coco_shape = [*mp_array.shape]
coco_shape[-2] = 18
coco_array = np.zeros(coco_shape)

In [60]:
mp_keys = [
    0,
    0,
    12,
    14,
    16,
    11,
    13,
    15,
    24,
    26,
    28,
    23,
    25,
    27,
    5,
    2,
    8,
    7,
]

In [72]:
coco_array[..., range(18), :] = mp_array[..., mp_keys, :]
coco_array[..., 1, :] = np.mean(mp_array[..., [11, 12], :], axis=-2)

In [73]:
coco_array[0]

array([[  0.,   1.,   2.],
       [115., 116., 117.],
       [120., 121., 122.],
       [140., 141., 142.],
       [160., 161., 162.],
       [110., 111., 112.],
       [130., 131., 132.],
       [150., 151., 152.],
       [240., 241., 242.],
       [260., 261., 262.],
       [280., 281., 282.],
       [230., 231., 232.],
       [250., 251., 252.],
       [270., 271., 272.],
       [ 50.,  51.,  52.],
       [ 20.,  21.,  22.],
       [ 80.,  81.,  82.],
       [ 70.,  71.,  72.]])